In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import os
from openpyxl.utils import get_column_letter

## Funciones primera parte del programa

In [2]:
def generarDataframePrincipal(rutaBalanzas):
    # Intentar leer el archivo Excel
    try:
        nombresHojasArchivo = pd.ExcelFile(rutaBalanzas).sheet_names
    except Exception as e:
        mensajeError = f"ERROR: No se pudo leer el archivo en ruta {rutaBalanzas}: {e}"
        return True, mensajeError, None, -1

    # Elegir una hoja válida
    while True:
        print(f'Meses contenidos: {", ".join(nombresHojasArchivo[:])}.')
        nombreHojaPorLeer = input("Introduzca mes a leer: ").strip()
        print(nombreHojaPorLeer)

        if nombreHojaPorLeer in nombresHojasArchivo:
            break
        else:
            print("Entrada inválida. Intente de nuevo.\n")

    # Intentar leer la hoja específica
    try:
        dfPrincipal = pd.read_excel(rutaBalanzas, sheet_name=nombreHojaPorLeer)
    except Exception as e:
        mensajeError = f"ERROR: No se puedo leer la hoja '{nombreHojaPorLeer}': {e}"
        return True, mensajeError, None, -1

    # Verificar que tenga 7 columnas
    if dfPrincipal.shape[1] != 7:
        mensajeError = f"ERROR: El archivo no tiene 7 columnas. Se encontraron {dfPrincipal.shape[1]} columnas."
        return True, mensajeError, None, -1

    # Todo salió bien
    print("EXITO: Se generó bien el dataframe inicial.")
    return False, '', dfPrincipal, nombreHojaPorLeer

#--------------------------------------------------------------------------------------#

def obtenerNumeroFilaAmarillaDataframe(rutaBalanzas, nombreHojaPorLeer, dfPrincipal):
    # Variables locales
    numeroFilaAmarillaExcel = -1
    numeroFilaAmarillaDf = -1
    contadorFilaAmarilla = 0
    intervalo = 4
    
    filasDf = dfPrincipal.shape[0]
    columnasDf = dfPrincipal.shape[1]
    
    esColorAmarillo = True
    sonCeldasNaN = True
    
    colorCeldaAmarilla = 'FFFFFF00'
    colorCeldaEstandar = '00000000'
    
    
    # Objeto para leer el archivo y manejar los colores de las celdas
    try:
        wbPrincipal = openpyxl.load_workbook(rutaBalanzas)
        wsPrincipal = wbPrincipal[nombreHojaPorLeer]
    except Exception as e:
        mensajeError = f"ERROR: No se puedo abrir archivo o leer hoja '{nombreHojaPorLeer}' con openpyxl: {str(e)}"
        return Truel, mensajeError, numeroFilaAmarillaDf

    # Bucle para encontrar el número de la fila amarilla, la cual separa los datos de interés
    for fila in wsPrincipal.iter_rows(min_row=1, max_row=filasDf, max_col=columnasDf):
        # En cada loop se vuelve a un True
        esColorAmarillo = True
        
        contadorFilaAmarilla = contadorFilaAmarilla + 1
        
        coloresCeldas = []
        
        # Recorrer celdas en fila para recuperar color en ellas
        for celda in fila:
            colorObjeto = celda.fill.fgColor # Color automático es '00000000' o 'None'
            # Ver si colorObjeto tiene atributo 'rgb'
            if colorObjeto.type == 'rgb' and colorObjeto.rgb is not None:
                colorCelda = colorObjeto.rgb
            else: 
                colorCelda = colorCeldaEstandar
            coloresCeldas.append(colorCelda)
        
        # Recorrer colores celdas en fila
        for colorCelda in coloresCeldas:
            if colorCelda != colorCeldaAmarilla:
                esColorAmarillo = False
        
        if esColorAmarillo:
            numeroFilaAmarillaExcel = contadorFilaAmarilla
            break
    
    # Verificar que se encontró fila amarilla en Excel 
    if numeroFilaAmarillaExcel == -1:
        mensajeError = f"ERROR: No se encontró fila amarilla. Revisar archivo en ruta: {rutaBalanzas}."
        return True, mensajeError, numeroFilaAmarillaDf
        
    # Verificar fila amarilla 2 posiciones antes en el DF
    for celda in dfPrincipal.iloc[numeroFilaAmarillaExcel - 2].tolist(): # Recorrer fila con iloc
        if not pd.isna(celda): # Verificar celda distinta de NaN
            sonCeldasNaN = False # Hay una celda distinta de NaN
            break
    
    #Verificar amarilla en un cierto intervalo alrededor de la fila amarilla en el Excel si no es dos posiciones antes
    if sonCeldasNaN:
        numeroFilaAmarillaDf = numeroFilaAmarillaExcel - 2 # La fila amarilla si está dos posiciones antes en el DF
    else:
        filasConNan = [] # Lista en caso de que haya más filas con NaN
        for i in range(-intervalo, intervalo+1):
            esFilaNaN = True
            for celda in dfPrincipal.iloc[numeroFilaAmarillaExcel + i].tolist():
                if not pd.isna(celda):
                    esFilaNaN = False
                    break
            if esFilaNaN: 
                # Agregar número fila NaN
                filasConNan.append(numeroFilaAmarillaExcel + i)

        # Verificar que solo hay una fila con NaN en ese intervalo (sería la amarilla en ese caso)
        if len(filasConNan) == 1:
            numeroFilaAmarillaDf = filasConNan[0]
        else:
            mensajeError = f"ERROR: Hay más de una fila vacía. No hay fila amarilla concluyente. Revisar archivo en ruta {rutaBalanzas}."
            return True, mensajeError, numeroFilaAmarillaDf
    
    # Todo salió bien
    print(f"EXITO: Se encontró fila amarilla en dataframe. Excel: {numeroFilaAmarillaExcel} | DF: {numeroFilaAmarillaDf}")
    return False,'', numeroFilaAmarillaDf

#--------------------------------------------------------------------------------------#

def obtenerNumeroFilaNombresColumnas(dfPrincipal):
    # Variables locales
    numeroFilaNombresColumnasDf = -1
    
    conjuntoEsperado = set(['CUENTA','NAT.','NOMBRE','SALDO INICIAL','CARGOS','ABONOS','SALDO FINAL'])

    # Bucle para encotrar fila que contiene los nombres reales de las columnas del dataframe final
    for indice, fila in dfPrincipal.iterrows():
        if conjuntoEsperado == set(fila.tolist()): 
            numeroFilaNombresColumnasDf = indice
            break
           
    if numeroFilaNombresColumnasDf == -1:
        mensajeError = "ERROR: La fila con los nombres de las columnas no se encontró."
        return True, mensajeError, numeroFilaNombresColumnasDf
    
    # Todo salió bien
    print("EXITO: Se obtuvo el número de la fila que contiene el nombre de las columnas que describen la info del documento.")
    return False, '', numeroFilaNombresColumnasDf
    
#--------------------------------------------------------------------------------------#

def obtenerConceptoGananciaCambiaria(dfPrincipal, numeroFilaNombresColumnasDf):
    try:
        # Variables locales
        conceptoGananciaCambiaria = 'GANANCIA CAMBIARIA'

        # Obtener nombres columnas del documento que describe la información (no son las verdaderas columnas del dataframe)
        nombresColumnas = dfPrincipal.iloc[numeroFilaNombresColumnasDf].tolist()

        # Obtener indice de la columna con los nombres (conceptos) y el de la columna de abonos
        indiceColumnaCuenta = nombresColumnas.index('CUENTA')
        indiceColumnaNombre = nombresColumnas.index('NOMBRE')
        indiceColumnaAbonos = nombresColumnas.index('ABONOS')

        # Obtener elementos de esa columna
        listaConceptosColumnaNombres = dfPrincipal.iloc[:, indiceColumnaNombre].tolist()

        if conceptoGananciaCambiaria not in listaConceptosColumnaNombres:
            mensajeError = "ERROR. No se encontró el concepto Ganancia Cambiaria. Revisar archivo Excel."
            return True, mensajeError, -1, conceptoGananciaCambiaria, -1

        # Obtener el índice del concepto de GANANCIA CAMBIARIA en la columna NOMBRES
        indiceConceptoGananciaCambiaria = listaConceptosColumnaNombres.index(conceptoGananciaCambiaria)

        # Obtener la cuetna y el abono de GANANCIA CAMBIARIA
        cuentaGananciaCambiaria = dfPrincipal.iloc[indiceConceptoGananciaCambiaria, indiceColumnaCuenta]
        abonoGananciaCambiaria = dfPrincipal.iloc[indiceConceptoGananciaCambiaria, indiceColumnaAbonos]

        # Todo salió bien
        print("EXITO: Se obtuvo el concepto Ganancia Cambiaria y su abono.")
        return False, '', [cuentaGananciaCambiaria, conceptoGananciaCambiaria, abonoGananciaCambiaria]
    
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', None

#--------------------------------------------------------------------------------------#

def crearDataframeFinalFiltrado(dfPrincipal, numeroFilaNombresColumnasDf, numeroFilaAmarillaDf):
    columnasAEliminar = ['SALDO INICIAL', 'ABONOS', 'SALDO FINAL', 'NAT.']

    try:
        filasDf = dfPrincipal.shape[0]
        
        # Seleccionar filas
        indicesParaGenerarDf = [numeroFilaNombresColumnasDf] + list(range(numeroFilaAmarillaDf + 1, filasDf))
        df = dfPrincipal.iloc[indicesParaGenerarDf].reset_index(drop=True)

        # Usar primera fila como nombres de columna
        df.columns = df.iloc[0]
        df = df.drop(index=0).reset_index(drop=True)

        # Eliminar filas vacías
        df = df[df['CUENTA'].notna()].reset_index(drop=True)

        # Eliminar columnas innecesarias
        dfFinal = df.drop(columns=columnasAEliminar, errors='ignore')

        print('EXITO: El dataframe final se generó.')
        return False, '', dfFinal

    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', None

#--------------------------------------------------------------------------------------#

def obtenerConceptosIndependientes(dfFinal_P1):
    conceptosIndependientes = ['GASTOS FINANCIEROS', 'COMISIONES BANCARIAS', 'PERDIDA CAMBIARIA']
    
    dic = {}
    
    filasConceptosIndependientes = []
    
    try:
        for concepto in conceptosIndependientes:
            listaFila = dfFinal_P1[dfFinal_P1['NOMBRE'] == concepto].index.tolist()

            if len(listaFila) != 1:
                mensajeError = f'ERROR: Concepto {concepto} no se encontró o se encontró en más de una fila en dfFinal_P1.' 
                return True, mensajeError, {}, None

            cuenta = dfFinal_P1.loc[listaFila[0], 'CUENTA']
            cargo = dfFinal_P1.loc[listaFila[0], 'CARGOS']
            filasConceptosIndependientes.append(listaFila[0])

            dic[concepto] = [cuenta, cargo]


        # Todo salió bien con el diccionario
        print(f'EXITO: Se recuperaron los conceptos independientes en diccionario retornado.')

        # Eliminar filas con los conceptos independientes del DF
        dfFinal_P1.drop(index=filasConceptosIndependientes, axis=0, inplace=True)
        dfFinal_P1.reset_index(drop=True, inplace=True)

        # Eliminar fila sin uso: 'GASTOS DE PRODUCCION'
        if dfFinal_P1.loc[0, 'NOMBRE'] == 'GASTOS DE PRODUCCION':
            dfFinal_P1.drop(index=0, axis=0, inplace=True)
            dfFinal_P1.reset_index(drop=True, inplace=True)
            print(f'EXITO: Se eliminó la fila innecesaria con nombre GASTOS DE PRODUCCION.')

        # Todo salió bien
        print(f'EXITO: Se eliminaron las filas con los conceptos independientes.')
        return False, '', dic, dfFinal_P1
    
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', None

## Funciones segunda parte del programa

In [3]:
def combinarConceptosIndependientes(dicConceptosIndependientes, listaGananciaCambiaria):
    try:
        # [cuentaGananciaCambiaria, conceptoGananciaCambiaria, abonoGananciaCambiaria]
        dicConceptosIndependientes[listaGananciaCambiaria[1]] = [listaGananciaCambiaria[0], listaGananciaCambiaria[2]]

        print('EXITO: Combinación de todos los conceptos independientes en un solo diccionario.')
        return False, '', dicConceptosIndependientes
    
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', {}

#--------------------------------------------------------------------------------------#

def obtenerFilaPadreHija(dfFinal_P1):
    cuentas = dfFinal_P1['CUENTA'].tolist()

    # Estos índices de la siguiente lista son clases 'padres', hay que eliminarlos. Los que hay que recuperar son los subconceptos
    filasCuentasPadre = []
    filasCuentasHija = []
    filasCuentasHijaHija = []
    filasCuentasPadreHija = []
    
    try:
        for i, cuenta in enumerate(cuentas):
            # Split cuenta concepto en guiones
            cuentaSplit = cuenta.strip().split('-')
            # Obtener grupos de 4 números contiene
            longitudCuenta = len(cuentaSplit)

            # Si se usa len(cuentas) sin -1 hay un error en los índices
            if i < len(cuentas)-1:    
                # Información de la cuenta siguiente
                cuentaSiguienteSplit = cuentas[i+1].strip().split('-')
                longitudCuentaSiguiente = len(cuentaSiguienteSplit)

                # Verificar si es subcuenta. En caso afirmativo, siempre hay un elemento más (otro grupo de 4), pero la raíz es la misma.
                if longitudCuenta < longitudCuentaSiguiente and cuentaSplit == cuentaSiguienteSplit[:-1]:
                    if longitudCuenta == 1:
                        filasCuentasPadre.append(i)
                        continue

                    if cuentaSplit[0] in dfFinal_P1.loc[filasCuentasPadre]['CUENTA'].tolist():
                        filasCuentasPadreHija.append(i)
                    else:
                        filasCuentasPadre.append(i)

                else:
                    # Una cuenta hija-hija SIEMPRE tiene 3 grupos de 4 números y es hija es una cuenta con 2 grupos de 4 números
                    if (longitudCuenta == 3) and ('-'.join(cuentaSplit[0:2]) in dfFinal_P1.loc[filasCuentasPadreHija]['CUENTA'].tolist()):
                        filasCuentasHijaHija.append(i)
                    else:
                        filasCuentasHija.append(i)
            elif i == len(cuentas)-1:
                # La última cuenta del loop no tiene subcuentas, se agrega directamente a la lista
                filasCuentasHija.append(i)

        print("EXITO: Se crearon las listas con filas de cuentas Padre, Hija, Hija-Hija y PadreHija.")
        return False, '', filasCuentasPadre, filasCuentasHija, filasCuentasHijaHija, filasCuentasPadreHija
    
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', [], [], [], []
    
#--------------------------------------------------------------------------------------#

def obtenerDicCCRC(dfFinal_P1, filasCuentasPadre, filasCuentasHija, filasCuentasHijaHija):
    
    dfHijas = dfFinal_P1.loc[filasCuentasHija + filasCuentasHijaHija]
    
    dicCuentasConceptoPadre = dict(zip(dfFinal_P1.loc[filasCuentasPadre]['CUENTA'], dfFinal_P1.loc[filasCuentasPadre]['NOMBRE']))

    conceptosRevisados = []

    # Concepto: [[cuenta1, rubro1, cantidad1], [cuenta2, rubro2, cantidad2], ..., [cuentaN, rubroN, cantidadN]]. 
    # CCRC=ConceptoCuentaRubroCantidad
    dicCCRC = {}
    

    try:
        # Recorrer filas a recuperar
        for concepto in dfHijas['NOMBRE'].tolist():
            # Si el concepto ya se obtuvo el loop cancela y se sigue al siguiente que es la siguiente fila
            if concepto in conceptosRevisados:
                continue

            # Se agrega a la lista de conceptos ya revisados
            conceptosRevisados.append(concepto)

            # Se crea llave de ese concepto con una lista vacía como valor
            dicCCRC[concepto] = []

            # Se obtiene la lista de las cuentas hija (son los conceptos que importan y a recuperar) que tienen ese concepto
            cuentasConConcepto = dfHijas[dfHijas['NOMBRE'] == concepto]['CUENTA'].tolist()   
            cargosConConcepto = dfHijas[dfHijas['NOMBRE'] == concepto]['CARGOS'].tolist()
            
            # Recorrer cuentas de las cuentas asociadas al concepto actual del loop
            for j, cuentaConConcepto in enumerate(cuentasConConcepto):
                longitudCCC = len(cuentaConConcepto.strip().split('-'))
                rubroPerteneciente = []
                
                # Obtener a cuántos rubros pertenece la cuenta acutal del loop (debe siempre pertenecer a uno)
                for cuentaPadre in dicCuentasConceptoPadre.keys():
                    if longitudCCC == 3:
                        if all(elem in cuentaConConcepto.strip().split('-')[:-1] for elem in cuentaPadre.strip().split('-')):
                            rubroPerteneciente.append(dicCambioNombreRubros[dicCuentasConceptoPadre[cuentaPadre]])

                    if longitudCCC == 2:
                        if all(elem in cuentaConConcepto.strip().split('-')[0] for elem in cuentaPadre.strip().split('-')):
                            rubroPerteneciente.append(dicCambioNombreRubros[dicCuentasConceptoPadre[cuentaPadre]])

                if len(rubroPerteneciente) != 1:
                    mensajeError = f'ERROR: La cuenta {cuentaConConcepto} del concepto {concepto} pertenece a más de un rubro.'
                    return True, mensajeError, {}

                dicCCRC[concepto].append([cuentaConConcepto, rubroPerteneciente[0], cargosConConcepto[j]])
               
        # Todo salió bien    
        print('EXITO: Se creo dicCCRC.')
        return False, '', dicCCRC
    
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', {}
    
#--------------------------------------------------------------------------------------#    

def separarConceptoPorClavesDicCCRC(listaConceptos, dicCCRC):
    try:
        for concepto in listaConceptos:
            # Separar concepto
            clave, valor = concepto, dicCCRC[concepto]

            for tercia in valor:
                ultimoGrupo = limpiarDigitos(tercia[0].strip().split('-')[-1])
                nuevaClave = f'{clave}-{ultimoGrupo}'

                # Crear nuevo elemento del diccionario con nueva clave
                if nuevaClave not in dicCCRC:
                    dicCCRC[nuevaClave] = []

                dicCCRC[nuevaClave].append([tercia[0], tercia[1], tercia[2]])

            # Eliminar elemento con clave del concepto
            del dicCCRC[concepto]

        print(f'EXITO: Separación por claves de los conceptos {", ".join(listaConceptos)}')
        return False, '', dicCCRC
        

    except Exception as e:
        return True, f'ERROR incesperado: {str(e)}', {}


In [4]:
def limpiarDigitos(digito):
    if not digito.isdigit():
        print("ERROR. El grupo debe contener solo dígitos")

    if len(digito) == 4:
        # 000A -> A (A ≠ 0)
        if digito[:3] == "000":
            return digito[3]
        # 00AA -> AA (primer A ≠ 0)
        elif digito[:2] == "00":
            return digito[2:]
        # 0AAA -> AAA (primer A ≠ 0)
        elif digito[0] == "0":
            return digito[1:]
        else:
            pass
    else:
        print("ERROR. El grupo debe tener exactamente 4 caracteres.")
        
#--------------------------------------------------------------------------------------#   
        
def obtenerDicCCRCCuentaDefinitiva(filasCuentasHijaHija, dicCCRC):
    try: 
        # Así deben quedar los elementos dl diccionario dicCCRCnuevaCuenta:
        # Conceptos: [cuentaGlobal, [rubro1, cantidad1], [rubro2, cantidad2], ..., [rubroN, cantidadN]]. 
        # CCRC=ConceptoCuentaRubroCantidad
        dicCCRCnuevaCuenta = {}
        
        cuentasHijaHija = dfFinal_P1.loc[filasCuentasHijaHija]['CUENTA'].tolist()

        for clave, valor in dicCCRC.items():
            # Ver si la CUENTA en un valor de la clave es hija de una cuenta padre-hija
            variedadCuentasClave = []

            # Crear clave con una lista con un elemento (valor por default de la cuenta definitiva) como valor
            dicCCRCnuevaCuenta[clave] = ['-1']

            # Obtener cuenta definitiva. tercia[0] es la cuenta, tercia[1] es el rubro y tercia[2] es la cantidad o cargo.
            for tercia in valor:
                if tercia[0] in cuentasHijaHija:
                    ultimoGrupo = limpiarDigitos(tercia[0].strip().split('-')[-2]) + '-' + limpiarDigitos(tercia[0].strip().split('-')[-1])
                else:
                    ultimoGrupo = limpiarDigitos(tercia[0].strip().split('-')[-1])

                if ultimoGrupo not in variedadCuentasClave:
                    variedadCuentasClave.append(ultimoGrupo)

                # Agregar [rubro, cargo] de la tercia actual en este loop
                dicCCRCnuevaCuenta[clave].append([tercia[1],tercia[2]])

            cuentaDefinitiva = ''
            
            if len(variedadCuentasClave) == 1:
                cuentaDefinitiva = variedadCuentasClave[0]
            if len(variedadCuentasClave) > 1:
                cuentaDefinitiva = '-'.join(variedadCuentasClave)

            # Cambiar el valor por default de la cuenta definitiva establecido en '-1'
            dicCCRCnuevaCuenta[clave][0] = cuentaDefinitiva
            
        # Todo salió bien
        print('EXITO: Se creo dicCCRCnuevaCuenta.')
        return False, '', dicCCRCnuevaCuenta
    
    except Exception as e:
        return True, f'ERROR incesperado: {str(e)}', {}
    
#--------------------------------------------------------------------------------------#   
   
def gasolinaEnFavorDeCombustibles(dicCCRCnuevaCuenta):
    try:
        if 'COMBUSTIBLES' in dicCCRCnuevaCuenta and 'GASOLINA' in dicCCRCnuevaCuenta:
            # Crear diccionario de rubros de GASOLINA para acceso rápido
            gasolinaLista = dicCCRCnuevaCuenta['GASOLINA']
            # Dicccionario con clave el rubro y valor el indice que comienza en 1
            gasolinaRubros = {rubro[0]: i for i, rubro in enumerate(gasolinaLista[1:], start=1)}

            for par in dicCCRCnuevaCuenta['COMBUSTIBLES'][1:]:
                rubro, valor = par

                if rubro in gasolinaRubros:
                    idx = gasolinaRubros[rubro]
                    gasolinaLista[idx][1] = round(gasolinaLista[idx][1] + valor, 2)
                else:
                    gasolinaLista.append([rubro, valor])

            print('EXITO: Suma de gastos para conceptos GASOLINA y COMBUSTIBLES.')

            del dicCCRCnuevaCuenta['COMBUSTIBLES']
            print('EXITO: Eliminación del concepto COMBUSTIBLES.')

        elif 'COMBUSTIBLES' in dicCCRCnuevaCuenta and 'GASOLINA' not in dicCCRCnuevaCuenta:
            dicCCRCnuevaCuenta['GASOLINA'] = dicCCRCnuevaCuenta.pop('COMBUSTIBLES')
            print('EXITO: Concepto COMBUSTIBLES sí existe pero GASOLINA no. Se hizo la copia para crear GASOLINA y se eliminó COMBUSTIBLES.')

        else: # O COMBUSTIBLES no existe o no existen ambos conceptos.
            pass
        
        # Todo salio bien
        return False, '', dicCCRCnuevaCuenta
    
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', {}

## Funciones tercera parte del programa

In [5]:
def crearDataFrameSinCalculos(dicDF_0, meses, datos_a_agregar, dicCCRCnuevaCuenta):
    # ------ 1 ------
    # Crear fila 0 con único valor en la columna CONCEPTO, las demás columnas tienen valor NaN
    try: 
        for clave, valor in dicDF_0.items():
            if clave == 'CONCEPTO':
                valor.append(meses[nombreHojaPorLeer])
            else:
                valor.append(np.nan)
            
    except:
        return True, f'Error inesperado en 1: {str(e)}', None
    
    
    # ------ 2 ------
    # Las filas siguientes tendran la info de las fijas hijas e hijas-hijas
    # Agregar filas con puro dato None. El número de filas a agregar es la cantidad de conceptos en dicCCRCnuevaCuenta.
    try:
        for valor in dicDF_0.values():
            for _ in range(len(dicCCRCnuevaCuenta)):
                valor.append(np.nan)
    except:
        return True, f'Error inesperado en 2: {str(e)}',None
    
    
    # ------ 3 ------
    # Llenar filas en las columnas con la información recuperada. Las demás ya tienen None por default
    try:
        fila = 0

        # Conceptos: [cuentaGlobal, [rubro1, cantidad1], [rubro2, cantidad2], ..., [rubroN, cantidadN]]. 
        for concepto, valorConcepto in dicCCRCnuevaCuenta.items():
            fila += 1

            # Las siguientes dos listas guardan elemts correspondientes: el í-esimo elem corresponde el i-ésimo elem de la otra
            # Las listas guardan la info de una fila en las columnas donde hay información
            columnasALlenar = ['CUENTA','CONCEPTO']
            datos = [valorConcepto[0], concepto]

            for par in valorConcepto[1:]:
                # Si la cantidad es 0, se agrega un valor NaN, si no se agrega el valor de la cantidad
                if par[1] != 0:
                    columnasALlenar.append(par[0])
                    datos.append(par[1])
                else:
                    columnasALlenar.append(par[0])
                    datos.append(np.nan)

            # Con el índice i recupero el elemento con el mismo índice (posición) de la lista datos
            for i, columna in enumerate(columnasALlenar):
                dicDF_0[columna][fila] = datos[i]
    
    except Exception as e:
        return True, f'ERROR inesperado en 3: {str(e)}', None
    
    # ------ 4 ------
    # Agregar datos independientes y otro más al final del dataframe
    try:
        for datos in datos_a_agregar:
            cuenta = datos['CUENTA']
            # Para COMISIONES BANCARIAS, aplicar transformación a la cuenta
            if datos['CONCEPTO'] == 'COMISIONES BANCARIAS':
                cuentaSplit = cuenta.strip().split('-')
                cuenta = cuentaSplit[0] + '-' + limpiarDigitos(cuentaSplit[1])

            # Agregar info a la filas de las columnas CUENTA, CONCEPTO y TOTAL GASTOS; las demás tienen valor NaN
            for clave in dicDF_0:
                if clave == 'CUENTA':
                    dicDF_0[clave].append(cuenta)
                elif clave == 'CONCEPTO':
                    dicDF_0[clave].append(datos['CONCEPTO'])
                elif clave == 'TOTAL GASTOS':
                    dicDF_0[clave].append(datos['TOTAL GASTOS'])
                else:
                    dicDF_0[clave].append(np.nan)
    except Exception as e:
        return True, f'ERROR inesperado en 4: {str(e)}', None
    
    # ------ 5 -----
    # Generar dataframe
    try:
        df = pd.DataFrame(dicDF_0)
    except Exception as e:
        return True, f'ERROR inesperado en 5: {str(e)}', None
    
    # Todo salió bien
    print('EXITO: Se generó el DataFrame final sin los cálculos pero con la información recuperada.')
    return False, '', df

#--------------------------------------------------------------------------------------#

def clave_ordenacion(x):
        return int(x.split('-')[0])
    
#--------------------------------------------------------------------------------------#

def ordenarDataframe(df, dicCCRCnuevaCuenta):
    try:
        # Reordenar filas de los conceptos hija por el número de la columna CUENTA dejando intactas las demás filas
        dfOrdenado = df.loc[1:len(dicCCRCnuevaCuenta)].sort_values(by='CUENTA', key=lambda col: col.map(clave_ordenacion))
        
        # Moficiar con dfOrdenado la parte del dataframe correspondiente a las filas entre 1 y len(dicCCRCnuevaCuenta)
        df.loc[1:len(dicCCRCnuevaCuenta)] = dfOrdenado.values
        
        # Todo salió bien
        print('EXITO: Se ordeno el DataFrame sin los cálculos.')
        return False, '', df
    
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', None

#--------------------------------------------------------------------------------------#

def calculosFilasColumnas(df, dicCCRCnuevaCuenta, colsRubros, colsGto, colsCta):
    # ------ 1 ------
    try:
        # Obtener TOTAL GASTOS para una misma fila de los conceptos hija
        # En el df, los conceptos hija se encuentran entre la fila 1 y len(dicCCRCnuevaCuenta)
        df.loc[1:len(dicCCRCnuevaCuenta), 'TOTAL GASTOS'] = df.loc[1:len(dicCCRCnuevaCuenta), colsRubros].sum(axis=1).round(2)
        
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', None
    # ------ 2 ------
    try:
        # Operaciones para obtener el TOTAL DEL MES de las columans correspondientes a los rubros y de TOTAL GASTOS
        # Operacioens en rubros
        for columna in colsRubros:
            df.loc[len(dicCCRCnuevaCuenta)+1, columna] = df.loc[1:len(dicCCRCnuevaCuenta), columna].sum().round(2)

        #Operacion en TOTAL GASTOS
        df.loc[len(dicCCRCnuevaCuenta)+1, 'TOTAL GASTOS'] = df.loc[1:len(dicCCRCnuevaCuenta), 'TOTAL GASTOS'].sum().round(2)
        
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', None
    
    # ------ 3 ------
    try:
        # Obtener porcentaje GTO y CTA
        #Porcentaje GTO
        for k,columna in enumerate(colsGto):
            for i in range(1,len(dicCCRCnuevaCuenta)+1):
                df.loc[i,columna] = ((df.loc[i,colsRubros[k]]/df.loc[len(dicCCRCnuevaCuenta)+1, colsRubros[k]])*100).round(2)

        #Porcentaje CTA
        for k,columna in enumerate(colsCta):
            for i in range(1,len(dicCCRCnuevaCuenta)+1):
                df.loc[i,columna] = ((df.loc[i,colsRubros[k]]/df.loc[i, 'TOTAL GASTOS'])*100).round(2)
                
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', None
            
    # ------ 4 ------
    try:
        # Obtener porcentaje columna TOTAL %
        for i in range(1, len(dicCCRCnuevaCuenta)+1):
            df.loc[i,'TOTAL %'] = ((df.loc[i,'TOTAL GASTOS']/df.loc[len(dicCCRCnuevaCuenta)+1,'TOTAL GASTOS'])*100).round(2)
    
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', None

    # ------ 5 ------
    try:
        # Obtener suma de los porcentages de cada columna GTO y en la columna TOTAL %
        # Columnas GTO
        for columna in colsGto:
            df.loc[len(dicCCRCnuevaCuenta)+1,columna] = df.loc[1:len(dicCCRCnuevaCuenta), columna].sum().round(1)

        #Columna TOTAL %
        df.loc[len(dicCCRCnuevaCuenta)+1,'TOTAL %'] = df.loc[1:len(dicCCRCnuevaCuenta),'TOTAL %'].sum().round(1)
    
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', None
    
    # ------ 6 ------
    try:
        # Obtener suma de los porcentajes para cada fila de los conceptos hija de las columnas CTA
        df.loc[1:len(dicCCRCnuevaCuenta),'EXTRA'] = df.loc[1:len(dicCCRCnuevaCuenta), colsCta].sum(axis=1).round(1)
        
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}', None
    
    # Todo salió bien
    print('EXITO: Cálculos en dataframe.')
    return False, '', df

## Funciones cuarta parte del programa

In [6]:
def guardarDataframeExcel(df, rutaSalida):
    try:
        # Verificamos si el archivo ya existe
        if os.path.exists(rutaSalida):
            # Abrimos el archivo existente y agregamos hoja nueva
            with pd.ExcelWriter(rutaSalida, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer, index=False, sheet_name=f'GTOS {nombreHojaPorLeer}')

                worksheet = writer.sheets[f'GTOS {nombreHojaPorLeer}']

                for i, col in enumerate(df.columns):
                    max_len = max(
                        df[col].astype(str).map(len).max(),
                        len(str(col))
                    )
                    col_letter = get_column_letter(i + 1)  # Excel es 1-indexado
                    worksheet.column_dimensions[col_letter].width = max_len + 4
        else:
            # Creamos un nuevo archivo
            with pd.ExcelWriter(rutaSalida, engine='openpyxl') as writer:
                df.to_excel(writer, index=False, sheet_name=f'GTOS {nombreHojaPorLeer}')

                worksheet = writer.sheets[f'GTOS {nombreHojaPorLeer}']            

                for i, col in enumerate(df.columns):
                    max_len = max(
                        df[col].astype(str).map(len).max(),
                        len(str(col))
                    )
                    col_letter = get_column_letter(i + 1)  # Excel es 1-indexado
                    worksheet.column_dimensions[col_letter].width = max_len + 4
        
        # Todo salió bien
        print(f'EXITO: Dataframe guardado en archivo Excel en ruta {rutaSalida}')
        return False, ''
                    
    except Exception as e:
        return True, f'ERROR inesperado: {str(e)}'

## Celda que corre el programa

In [7]:
# Ruta a archivo balanza
rutaBalanzas = '.\BALANZAS 2025 gastos.xlsx'
# Archivo salida
rutaGastos = '.\GASTOS 2025.xlsx'

##################################### PRIMERA PARTE #####################################

while True:
    esErrorGlobal, mensajeError, dfPrincipal, nombreHojaPorLeer = generarDataframePrincipal(rutaBalanzas=rutaBalanzas)
    
    if esErrorGlobal:
        print(mensajeError)
        break
        
    esErrorGlobal, mensajeError, numeroFilaAmarillaDf = obtenerNumeroFilaAmarillaDataframe(rutaBalanzas=rutaBalanzas, 
                                                                                           nombreHojaPorLeer=nombreHojaPorLeer, 
                                                                                           dfPrincipal=dfPrincipal)
    if esErrorGlobal:
        print(mensajeError)
        break
        
    esErrorGlobal, mensajeError, numeroFilaNombresColumnasDf = obtenerNumeroFilaNombresColumnas(dfPrincipal=dfPrincipal)
    
    if esErrorGlobal:
        print(mensajeError)
        break
        
    esErrorGlobal, mensajeError, listaGananciaCambiaria = obtenerConceptoGananciaCambiaria(dfPrincipal=dfPrincipal, 
                                                                                           numeroFilaNombresColumnasDf=numeroFilaNombresColumnasDf)
    if esErrorGlobal:
        print(mensajeError)
        break
        
    esErrorGlobal, mensajeError, dfFinal_P1 = crearDataframeFinalFiltrado(dfPrincipal=dfPrincipal,
                                                                           numeroFilaNombresColumnasDf=numeroFilaNombresColumnasDf, 
                                                                           numeroFilaAmarillaDf=numeroFilaAmarillaDf)
    if esErrorGlobal:
        print(mensajeError)
        break
        
    esErrorGlobal, mensajeError, dicConceptosIndependientes, dfFinal_P1 = obtenerConceptosIndependientes(dfFinal_P1=dfFinal_P1)
    
    if esErrorGlobal:
        print(mensajeError)
        break
    
    # Todo salió bien
    print("\n---- EXITO en primera parte del programa ----\n")
    break
    
    
##################################### SEGUNDA PARTE #####################################

while(True):
    dicCambioNombreRubros = {'PLANTA DE ETIQUETAS':'PLANTA 1', 
                             'PLANTA DE LITOGRAFIA':'PLANTA 3', 
                             'P-4 LITOGRAFIA':'PLANTA 4', 
                             'MANTENIMIENTO DE TALLER':'MTO TALLER', 
                             'MANTENIMIENTO DEL EDIFICIO':'MTO EDIFICIO', 
                             'ALMACEN Y COMPRAS':'ALMACEN COMPRAS', 
                             'DISEÑO':'DISEÑO', 
                             'CALIDAD':'CALIDAD', 
                             'FACTURACION Y EMBARQUES':'FACT Y EMBARQ', 
                             'GASTOS DE VENTA':'VENTAS', 
                             'GASTOS DE ADMINISTRACION':'ADMON'}
    
    esErrorGlobal, mensajeError, dicConceptosIndependientes = combinarConceptosIndependientes(dicConceptosIndependientes=dicConceptosIndependientes,
                                                                listaGananciaCambiaria=listaGananciaCambiaria)
    
    if esErrorGlobal:
        print(mensajeError)
        break
    
    esErrorGlobal, mensajeError, filasCuentasPadre, filasCuentasHija, filasCuentasHijaHija, filasCuentasPadreHija = obtenerFilaPadreHija(dfFinal_P1=dfFinal_P1)
    
    if esErrorGlobal:
        print(mensajeError)
        break
    
    esErrorGlobal, mensajeError, dicCCRC = obtenerDicCCRC(dfFinal_P1=dfFinal_P1, 
                                                          filasCuentasPadre=filasCuentasPadre, 
                                                          filasCuentasHija=filasCuentasHija, 
                                                          filasCuentasHijaHija=filasCuentasHijaHija)
    
    if esErrorGlobal:
        print(mensajeError)
        break
        
    listaConceptos = ['VARIOS']    
    esErrorGlobal, mensajeError, dicCCRC = separarConceptoPorClavesDicCCRC(listaConceptos=listaConceptos, dicCCRC=dicCCRC)
    
    if esErrorGlobal:
        print(mensajeError)
        break
        
    esErrorGlobal, mensajeError, dicCCRCnuevaCuenta = obtenerDicCCRCCuentaDefinitiva(filasCuentasHijaHija=filasCuentasHijaHija,
                                                                                    dicCCRC=dicCCRC)
    
    if esErrorGlobal:
        print(mensajeError)
        break
        
    esErrorGlobal, mensajeError, dicCCRCnuevaCuenta = gasolinaEnFavorDeCombustibles(dicCCRCnuevaCuenta=dicCCRCnuevaCuenta)
    
    if esErrorGlobal:
        print(mensajeError)
        break
    
    # Todo salió bien
    print("\n---- EXITO en segunda parte del programa ----\n")
    break
    
    
##################################### TERCERA PARTE #####################################

while (True):
    dicDF_0 = {'FOLIOi':[], 'CUENTA':[], 'CONCEPTO':[], 'GTO1':[], 'PLANTA 1':[], 'CTA1':[], 'GTO2':[], 'PLANTA 3':[], 'CTA2':[],
             'GTO3':[], 'PLANTA 4':[], 'CTA3':[], 'GTO4':[], 'MTO TALLER':[], 'CTA4':[], 'GTO5':[], 'MTO EDIFICIO':[], 'CTA5':[], 
             'GTO6':[], 'ALMACEN COMPRAS':[], 'CTA6':[], 'GTO7':[], 'DISEÑO':[], 'CTA7':[], 'GTO8':[], 'CALIDAD':[], 'CTA8':[],
             'GTO9':[], 'FACT Y EMBARQ':[], 'CTA9':[], 'GTO10':[], 'VENTAS':[], 'CTA10':[], 'GTO11':[], 'ADMON':[], 'CTA11':[],
             'TOTAL GASTOS':[], 'TOTAL %':[], 'FOLIOf':[], 'EXTRA':[]}

    meses = {'ENE':'ENERO', 'FEB':'FEBRERO', 'MAR':'MARZO', 'ABR':'ABRIL', 'MAY':'MAYO', 'JUN':'JUNIO', 'JUL':'JULIO', 
             'AGO':'AGOSTO', 'SEPT':'SEPTIEMBRE', 'OCT':'OCTUBRE', 'NOV':'NOVIEMBRE', 'DIC':'DICIEMBRE'}

    datos_a_agregar = [
        {
            'CUENTA': np.nan,
            'CONCEPTO': f'TOTAL MES {meses[nombreHojaPorLeer]} 2025',
            'TOTAL GASTOS': np.nan
        },
        {
            'CUENTA': np.nan,
            'CONCEPTO': f'RENTA NO PAGADA (DEL MES {nombreHojaPorLeer})',
            'TOTAL GASTOS': np.nan
        },
        {
            'CUENTA': dicConceptosIndependientes['GASTOS FINANCIEROS'][0],
            'CONCEPTO': 'GASTOS FINANCIEROS',
            'TOTAL GASTOS': dicConceptosIndependientes['GASTOS FINANCIEROS'][1]
        },
        {
            'CUENTA': dicConceptosIndependientes['COMISIONES BANCARIAS'][0],
            'CONCEPTO': 'COMISIONES BANCARIAS',  
            'TOTAL GASTOS': dicConceptosIndependientes['COMISIONES BANCARIAS'][1]
        },
        {
            'CUENTA': '5404-3',
            'CONCEPTO': 'INTS PAG A INSTITUCIONES FINANC',  
            'TOTAL GASTOS': np.nan
        },
        {
            'CUENTA': '5404-4',
            'CONCEPTO': 'INTS PAGADOS A PF Y PM',  
            'TOTAL GASTOS': np.nan
        },
        {
            'CUENTA': dicConceptosIndependientes['PERDIDA CAMBIARIA'][0],
            'CONCEPTO': 'PERDIDA CAMBIARIA',  
            'TOTAL GASTOS': dicConceptosIndependientes['PERDIDA CAMBIARIA'][1]
        },
        {
            'CUENTA': dicConceptosIndependientes['GANANCIA CAMBIARIA'][0],
            'CONCEPTO': 'GANANCIA CAMBIARIA',  
            'TOTAL GASTOS': dicConceptosIndependientes['GANANCIA CAMBIARIA'][1]
        }
    ]
    
    colsRubros = ['PLANTA 1','PLANTA 3','PLANTA 4','MTO TALLER','MTO EDIFICIO','ALMACEN COMPRAS','DISEÑO','CALIDAD',
                   'FACT Y EMBARQ','VENTAS','ADMON']
    
    colsGto =  ['GTO1','GTO2','GTO3','GTO4','GTO5','GTO6','GTO7','GTO8','GTO9','GTO10','GTO11']
    
    colsCta =  ['CTA1','CTA2','CTA3','CTA4','CTA5','CTA6','CTA7','CTA8','CTA9','CTA10','CTA11']
    
    esErrorGlobal, mensajeError, df = crearDataFrameSinCalculos(dicDF_0=dicDF_0, meses=meses,
                                                                     datos_a_agregar=datos_a_agregar, 
                                                                     dicCCRCnuevaCuenta= dicCCRCnuevaCuenta)
        
    if esErrorGlobal:
        print(mensajeError)
        break

    esErrorGlobal, mensajeError, df = ordenarDataframe(df=df, dicCCRCnuevaCuenta=dicCCRCnuevaCuenta)
    
    if esErrorGlobal:
        print(mensajeError)
        break
        
    esErrorGlobal, mensajeError, df = calculosFilasColumnas(df=df, dicCCRCnuevaCuenta=dicCCRCnuevaCuenta, 
                                                            colsRubros=colsRubros, colsGto=colsGto,
                                                            colsCta=colsCta)
     
    if esErrorGlobal:
        print(mensajeError)
        break
        
    # Todo salió bien
    print("\n---- EXITO en tercera parte del programa ----\n")
    break
    
##################################### CUARTA PARTE #####################################

esErrorGlobal, mensajeError = guardarDataframeExcel(df, rutaSalida=rutaGastos)
    
if esErrorGlobal:
    print(mensajeError)
else: 
    # Todo el programa salió bien
    print("\n****** EXITO EN EL PROGRAMA (checar archivo) ******\n")

Meses contenidos: ENE, FEB, MAR, ABR, MAY, JUN, JUL, AGO, SEPT, modificar encabezado.
Introduzca mes a leer: SEPT
SEPT
EXITO: Se generó bien el dataframe inicial.
EXITO: Se encontró fila amarilla en dataframe. Excel: 267 | DF: 265
EXITO: Se obtuvo el número de la fila que contiene el nombre de las columnas que describen la info del documento.
EXITO: Se obtuvo el concepto Ganancia Cambiaria y su abono.
EXITO: El dataframe final se generó.
EXITO: Se recuperaron los conceptos independientes en diccionario retornado.
EXITO: Se eliminó la fila innecesaria con nombre GASTOS DE PRODUCCION.
EXITO: Se eliminaron las filas con los conceptos independientes.

---- EXITO en primera parte del programa ----

EXITO: Combinación de todos los conceptos independientes en un solo diccionario.
EXITO: Se crearon las listas con filas de cuentas Padre, Hija, Hija-Hija y PadreHija.
EXITO: Se creo dicCCRC.
EXITO: Separación por claves de los conceptos VARIOS
EXITO: Se creo dicCCRCnuevaCuenta.
EXITO: Suma de gas